<a href="https://colab.research.google.com/github/Xifus/SA-lab/blob/master/Sensitivity_Analysis_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install SAlib

We will work on Python during this Lab.

SALib is an open source library written in Python for performing sensitivity analysis. A typical sensitivity analysis using SALib follows four steps:
1. Determine the model inputs (parameters) and their sample range
2. Run the sample function to generate the model inputs
3. Evaluate the model using the generated inputs, saving the model outputs
4. Run the analyze function on the outputs to compute the sensitivity indices

SALib is the main package that we will play around today. With the help of google colab, the SALib package is already installed and ready to run.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

One of the important things in python is that you have to import the corresponding functions first before you can use them. Numpy provides basic built in functions (random number generator in this case), and Matplotlib provides the ability to show figures.

We import different sampling techniques ***sample***, sensitivity analysis method ***analyze*** and test functions ***test_functions*** from SALib package.

In [0]:
from SALib.sample import latin
from SALib.sample import sobol_sequence
from SALib.sample import morris as morris_samp
from SALib.sample import saltelli

In [0]:
from SALib.analyze import sobol
from SALib.analyze import morris

In [0]:
from SALib.test_functions import Ishigami
from SALib.test_functions import Sobol_G

Scatter plots for Random, Latin Hypercube, and Sobol' sequence. The default sample size (N) is 100. You can change the sample size and observe the scatter plots.  

In [0]:
problem ={'num_vars': 2,
          'names': ['x1', 'x2'],
          'bounds': [[0, 1],
                     [0, 1]]}
N = 100

Random sampling

In [0]:
rand_samp = np.random.rand(N, 2)
plt.figure()
plt.scatter(rand_samp[:,0], rand_samp[:, 1])
plt.title('Random')
plt.show()

Latin Hypercube

In [0]:
latin_samp = latin.sample(problem, N)
plt.figure()
plt.scatter(latin_samp[:,0], latin_samp[:, 1])
plt.title('Latin Hypercube')
plt.show()

Sobol' sequence

In [0]:
sobol_samp = sobol_sequence.sample(N, 2)
plt.figure()
plt.scatter(sobol_samp[:,0], sobol_samp[:, 1])
plt.title('Sobol\' sequence')
plt.show()

Put all three sampling methods together

In [0]:
plt.figure(figsize = (10, 10))
plt.scatter(rand_samp[:,0], rand_samp[:, 1], c = 'r', label = 'rand')
plt.scatter(latin_samp[:,0], latin_samp[:, 1], c = 'b', label = 'latin')
plt.scatter(sobol_samp[:,0], sobol_samp[:, 1], c = 'k', label = 'sobol')
plt.legend(loc = 'best')
plt.show()

Next is an example of Monte Carlo on approximating the value of Pi by using different sampling techniques. You can comment (#) or uncomment (remove # at the beginning of the line) to use different sampling techniques. How close can you get? How does the number of samples affect your approximation?

In [0]:
N_sample = 1000
MC_rand = np.random.rand(N_sample, 2)
#MC_latin = latin.sample(problem, N_sample)
#MC_sobol = sobol_sequence.sample(N_sample, 2)
#MC_saltelli = saltelli.sample(problem, N_sample)
count = 0
for i in range(N_sample):
    if MC_rand[i, 0]**2 + MC_rand[i, 1]**2 < 1.0:
        count += 1
pi_approx = count/N_sample * 4
print('real pi = ', np.pi)
print('approximated = ', pi_approx)

**Sensitivity analysis method in application**

After you defining the model inputs (ex. problem = {'num_vars', 'names', 'bounds'}), the workflow looks like this:


1.   Generate Samples (ex. param values = latin.sample(problem, 1000))
2.   Run the Model (ex. Y = Ishigami.evaluate(param values))
3.   Perform Analysis (ex. Si = sobol.analyze(problem, Y))




First, we use Ishigami Homma functinon 

![](https://raw.githubusercontent.com/Xifus/SA-lab/master/ishigami.PNG)

In [0]:
ishigami_problem ={
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

In [0]:
X_morris = morris_samp.sample(ishigami_problem, 1000, num_levels=4)
Y_morris = Ishigami.evaluate(X_morris)
Si_morris = morris.analyze(ishigami_problem, X_morris, Y_morris, conf_level=0.95, print_to_console=True, num_levels=4)

**Morris Method: Perform Morris Analysis on model outputs**

---
SALib.analyze.morris.analyze(problem, X, Y, num_resamples = 1000,
conf_level = 0.95, print_to_console = False, num_levels = 4)

---




**Parameters**:

problem (dict) – The problem definition

X (numpy.matrix) – The NumPy matrix containing the model inputs
of dtype=float

X (numpy.matrix) – The NumPy matrix containing the model inputs
of dtype=float

num resamples (int) – The number of resamples used to compute the
confidence intervals (default 1000)

conf level (float) – The confidence interval level (default 0.95)

print to console (bool) – Print results directly to console (default
False)

num levels (int) – The number of grid levels, must be identical to the
value passed to SALib.sample.morris (default 4)

**Returns**:

Si – A dictionary of sensitivity indices containing the following
entries.

mu - the mean elementary effect

mu star - the absolute of the mean elementary effect

sigma - the standard deviation of the elementary effect

mu star conf - the bootstrapped confidence interval

names - the names of the parameters

In [0]:
X_sobol = latin.sample(ishigami_problem, 1000)
Y_sobol = Ishigami.evaluate(X_sobol)
Si_sobol = sobol.analyze(ishigami_problem, Y_sobol, print_to_console=True)

**Sobol’ Method: Perform Sobol’ Analysis on model outputs**


---

SALib.analyze.sobol.analyze(problem, Y, calc_second_order=True,
num_resamples=100, conf_level=0.95, print_to_console=False)

---



**Parameters**:

problem (dict) – The problem definition

Y (numpy.array) – A NumPy array containing the model outputs

calc second order (bool) – Calculate second-order sensitivities (default True)

num resamples (int) – The number of resamples (default 100)

conf level (float) – The confidence interval level (default 0.95)

print to console (bool) – Print results directly to console (default
False)

**Returns**: 

Si – A dictionary of sensitivity indices containing the following
entries.

S1 - the first order sensitivity

S1 conf - the confidence interval of the first order sensitivity with a
confidence level of 95%

ST - the total effect sensitivity

ST conf - the confidence interval of the total effect sensitivity with a
confidence level of 95%

S2 - the second order sensitivity (if calc second order is True)

S2 conf - the confidence interval of second order sensitivity with a
confidence level of 95% (if calc second order is True)

In [0]:
X_sobol_sal = saltelli.sample(ishigami_problem, 1000)
Y_sobol_sal = Ishigami.evaluate(X_sobol_sal)
Si_sobol_sal = sobol.analyze(ishigami_problem, Y_sobol_sal, print_to_console=True)

Now, we use V-function (Sobol g-function) and try the different methods.

![](https://raw.githubusercontent.com/Xifus/SA-lab/master/sobolG.PNG)

In [0]:
V_problem ={'num_vars': 8,
          'names': ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'],
          'bounds': [[0, 1]]*8}

In [0]:
X_morris_V = morris_samp.sample(V_problem, 999, num_levels=4)
Y_morris_V = Sobol_G.evaluate(X_morris_V)
Si_morris_V = morris.analyze(V_problem, X_morris_V, Y_morris_V, conf_level=0.95, print_to_console=True, num_levels=4)

In [0]:
X_sobol_V = latin.sample(V_problem, 990)
Y_sobol_V = Sobol_G.evaluate(X_sobol_V)
Si_sobol_V = sobol.analyze(V_problem, Y_sobol_V, print_to_console=True)

In [0]:
X_sobol_V_sal = saltelli.sample(V_problem, 1000)
Y_sobol_V_sal = Sobol_G.evaluate(X_sobol_V_sal)
Si_sobol_V_sal = sobol.analyze(V_problem, Y_sobol_V_sal, print_to_console=True)

**Convergence**

Try different samples and see how good the convergence is. How many samples do you need to get a reliable results? What is the accuracy?

Use the knowledge from previous steps and try to use different sampling techniques/sensitivity analysis method/sensitivity measures.

For example, changing Saltelli sampling method to Latin Hypercube; changing Sobol method to Morris method; changing first order to total effect.

Note: Morris method uses its own sampling method.

In [0]:
N_samples = [1000, 5000, 10000, 50000, 100000, 500000]

In [0]:
var_num = 3

In [0]:
S1_cache = np.zeros((len(N_samples), var_num))
ST_cache = S1_cache.copy()

In [0]:
for i in range(len(N_samples)):
    X_sobol_N = saltelli.sample(ishigami_problem, N_samples[i])
    Y_sobol_N = Ishigami.evaluate(X_sobol_N)
    Si_sobol_N = sobol.analyze(ishigami_problem, Y_sobol_N, print_to_console=False)
    S1_cache[i, :] = Si_sobol_N['S1']
    ST_cache[i, :] = Si_sobol_N['ST']

In [0]:
plt.figure()
plt.plot(S1_cache[:, 0])
plt.axhline(y = 0.3139, c = 'r', linestyle = '--', label = 'exact value')
plt.legend(loc = 'best')
plt.xticks(np.arange(len(N_samples)), N_samples)
plt.show()

Here are the exact values for first order and total effect sensitivity indices for reference.

***Ishigami Homma Function***

![](https://raw.githubusercontent.com/Xifus/SA-lab/master/ishigami_sobol.PNG)


***V-Fnction (Sobol g-Function)***

![](https://raw.githubusercontent.com/Xifus/SA-lab/master/gsobol_sobol.PNG)